In [17]:
from typing import Annotated

from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import BaseMessage
from typing_extensions import TypedDict

from langgraph.checkpoint.sqlite import SqliteSaver
from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode
from langchain_openai import AzureChatOpenAI
import os
from langgraph.prebuilt import ToolNode, tools_condition

class State(TypedDict):
    messages: Annotated[list, add_messages]


graph_builder = StateGraph(State)

os.environ["TAVILY_API_KEY"] = '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'
tool = TavilySearchResults(max_results=2)
tools = [tool]

APIKEY = os.environ.get('AZURE_OPENAI_4o_KEY1')
ENDPOINT = os.environ.get('AZURE_ENDPOINT_BMD2')
llm = AzureChatOpenAI(
    openai_api_version="2024-02-15-preview",
    deployment_name='gpt-4o',
    azure_endpoint=ENDPOINT,
    api_key=APIKEY,
)
llm_with_tools = llm.bind_tools(tools)


def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}


graph_builder.add_node("chatbot", chatbot)

tool_node = ToolNode(tools=[tool])
graph_builder.add_node("tools", tool_node)

graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)
graph_builder.add_edge("tools", "chatbot")
graph_builder.set_entry_point("chatbot")

memory = SqliteSaver.from_conn_string(":memory:")
graph = graph_builder.compile(
    checkpointer=memory,
    # This is new!
    interrupt_before=["tools"],
    # Note: can also interrupt __after__ actions, if desired.
    # interrupt_after=["tools"]
)

user_input = "I'm learning LangGraph. Could you do some research on it for me?"
config = {"configurable": {"thread_id": "1"}}
# The config is the **second positional argument** to stream() or invoke()!
events = graph.stream(
    {"messages": [("user", user_input)]}, config, stream_mode="values"
)
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()

================================ Human Message =================================

I'm learning LangGraph. Could you do some research on it for me?
================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (call_ttkggTOyRHR0vX4F5xpQqIwe)
 Call ID: call_ttkggTOyRHR0vX4F5xpQqIwe
  Args:
    query: LangGraph programming language
  tavily_search_results_json (call_9inlEVhYHFBDfJum13SgoVXM)
 Call ID: call_9inlEVhYHFBDfJum13SgoVXM
  Args:
    query: LangGraph tutorial


In [18]:
snapshot = graph.get_state(config)
snapshot.next

('tools',)

In [19]:
existing_message = snapshot.values["messages"][-1]
existing_message.tool_calls

[{'name': 'tavily_search_results_json',
  'args': {'query': 'LangGraph programming language'},
  'id': 'call_ttkggTOyRHR0vX4F5xpQqIwe'},
 {'name': 'tavily_search_results_json',
  'args': {'query': 'LangGraph tutorial'},
  'id': 'call_9inlEVhYHFBDfJum13SgoVXM'}]

In [12]:
# `None` will append nothing new to the current state, letting it resume as if it had never been interrupted
events = graph.stream(None, config, stream_mode="values")
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()

================================= Tool Message =================================
Name: tavily_search_results_json

[{"url": "https://langchain-ai.github.io/langgraph/tutorials/", "content": "Basic Reflection: Prompting the agent to reflect on and revise its outputs. Reflexion: Critiquing missing and superfluous details to guide next steps. Language Agent Tree Search: Using reflection and rewards to drive a tree search over agents. Self-Discovering Agent: Analyzing an agent that learns about its own capabilities."}, {"url": "https://medium.com/@cplog/introduction-to-langgraph-a-beginners-guide-14f9be027141", "content": "Feb 15, 2024. --. 2. LangGraph is a powerful tool for building stateful, multi-actor applications with Large Language Models (LLMs). It extends the LangChain library, allowing you to coordinate ..."}]
================================== Ai Message ==================================

### Overview of LangGraph

1. **Medium Article**: [Introduction to LangGraph: A Beginner's

In [21]:
existing_message.tool_calls

[{'name': 'tavily_search_results_json',
  'args': {'query': 'LangGraph programming language'},
  'id': 'call_ttkggTOyRHR0vX4F5xpQqIwe'},
 {'name': 'tavily_search_results_json',
  'args': {'query': 'LangGraph tutorial'},
  'id': 'call_9inlEVhYHFBDfJum13SgoVXM'}]

In [22]:
from langchain_core.messages import AIMessage, ToolMessage
answer = (
    "LangGraph is a library for building stateful, multi-actor applications with LLMs."
)
new_messages = [
    # The LLM API expects some ToolMessage to match its tool call. We'll satisfy that here.
    ToolMessage(content=answer, tool_call_id=existing_message.tool_calls[0]["id"]),
    # And then directly "put words in the LLM's mouth" by populating its response.
    AIMessage(content=answer),
]

new_messages[-1].pretty_print()


================================== Ai Message ==================================

LangGraph is a library for building stateful, multi-actor applications with LLMs.


In [23]:
graph.update_state(
    # Which state to update
    config,
    # The updated values to provide. The messages in our `State` are "append-only", meaning this will be appended
    # to the existing state. We will review how to update existing messages in the next section!
    {"messages": new_messages},
)

print("\n\nLast 2 messages;")
print(graph.get_state(config).values["messages"][-2:])

# graph.update_state(
#     config,
#     {"messages": [AIMessage(content="I'm an AI expert!")]},
#     # Which node for this function to act as. It will automatically continue
#     # processing as if this node just ran.
#     as_node="chatbot",
# )



Last 2 messages;
[ToolMessage(content='LangGraph is a library for building stateful, multi-actor applications with LLMs.', id='a13d310e-786f-49be-9a60-7cdd2d463c46', tool_call_id='call_ttkggTOyRHR0vX4F5xpQqIwe'), AIMessage(content='LangGraph is a library for building stateful, multi-actor applications with LLMs.', id='e020a0fd-1048-48b1-ba09-0a6125fc932d')]


In [28]:
graph.update_state(
    config,
    {"messages": [AIMessage(content="I'm an AI expert!")]},
    # Which node for this function to act as. It will automatically continue
    # processing as if this node just ran.
    as_node="chatbot",
)

{'configurable': {'thread_id': '1',
  'thread_ts': '1ef2fa68-6513-6b44-8003-6f90ed56b570'}}

In [30]:
snapshot = graph.get_state(config)
print(snapshot.values["messages"][-3:])
print(snapshot.next)

[ToolMessage(content='LangGraph is a library for building stateful, multi-actor applications with LLMs.', id='a13d310e-786f-49be-9a60-7cdd2d463c46', tool_call_id='call_ttkggTOyRHR0vX4F5xpQqIwe'), AIMessage(content='LangGraph is a library for building stateful, multi-actor applications with LLMs.', id='e020a0fd-1048-48b1-ba09-0a6125fc932d'), AIMessage(content="I'm an AI expert!", id='0fb8d508-5846-4785-bc36-0de0e9ee654f')]
()


new thread

In [31]:
user_input = "I'm learning LangGraph. Could you do some research on it for me?"
config = {"configurable": {"thread_id": "2"}}  # we'll use thread_id = 2 here
events = graph.stream(
    {"messages": [("user", user_input)]}, config, stream_mode="values"
)
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()

================================ Human Message =================================

I'm learning LangGraph. Could you do some research on it for me?
================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (call_HMdHS8gqVRh1WVt8Bz0pWrzM)
 Call ID: call_HMdHS8gqVRh1WVt8Bz0pWrzM
  Args:
    query: LangGraph programming language
  tavily_search_results_json (call_v6SeDbdhzrJJT9Jz3cTY8aaj)
 Call ID: call_v6SeDbdhzrJJT9Jz3cTY8aaj
  Args:
    query: LangGraph language tutorials


In [32]:
from langchain_core.messages import AIMessage

snapshot = graph.get_state(config)
existing_message = snapshot.values["messages"][-1]
print("Original")
print("Message ID", existing_message.id)
print(existing_message.tool_calls[0])
new_tool_call = existing_message.tool_calls[0].copy()
new_tool_call["args"]["query"] = "LangGraph human-in-the-loop workflow"
new_message = AIMessage(
    content=existing_message.content,
    tool_calls=[new_tool_call],
    # Important! The ID is how LangGraph knows to REPLACE the message in the state rather than APPEND this messages
    id=existing_message.id,
)

print("Updated")
print(new_message.tool_calls[0])
print("Message ID", new_message.id)
graph.update_state(config, {"messages": [new_message]})

print("\n\nTool calls")
graph.get_state(config).values["messages"][-1].tool_calls

Original
Message ID run-355b32b3-470a-49eb-aab6-2b5778c7fd6c-0
{'name': 'tavily_search_results_json', 'args': {'query': 'LangGraph programming language'}, 'id': 'call_HMdHS8gqVRh1WVt8Bz0pWrzM'}
Updated
{'name': 'tavily_search_results_json', 'args': {'query': 'LangGraph human-in-the-loop workflow'}, 'id': 'call_HMdHS8gqVRh1WVt8Bz0pWrzM'}
Message ID run-355b32b3-470a-49eb-aab6-2b5778c7fd6c-0


Tool calls


[{'name': 'tavily_search_results_json',
  'args': {'query': 'LangGraph human-in-the-loop workflow'},
  'id': 'call_HMdHS8gqVRh1WVt8Bz0pWrzM'}]

In [34]:
graph.get_state(config).values["messages"][-1]

AIMessage(content='', id='run-355b32b3-470a-49eb-aab6-2b5778c7fd6c-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'LangGraph human-in-the-loop workflow'}, 'id': 'call_HMdHS8gqVRh1WVt8Bz0pWrzM'}])